## RQ1 Forest Definition Comparison 

[Add description]

Steps
1. Load forest definitions maps from Step 1 AND Step 2
2. Reclassify all maps to FNF
3. Run comparative statistics
4. Sum pixels to create definition consensus map (Sexton et al 2016 approach)
5. Generate visualisations

Maybe useful for reclassifying raster? https://pygis.io/docs/e_raster_replace_values.html 

Maybe useful for the analysis part: zonal statistics https://github.com/shakasom/zonalstatistics

or here (scroll down to "Extraction with polygons"): https://py.geocompx.org/05-raster-vector 